### Imports and connect to my gDrive

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.io as sio
import seaborn as sn
from sklearn.metrics import confusion_matrix
import pandas as pd
import time 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
import scipy.io as sio
import copy
import pickle
import numpy as np
from scipy.signal import find_peaks
from os.path import dirname, join as pjoin
import datetime
import csv
import math
import sys

output_directory = "./drive/MyDrive/NSF_Research/NN/turb_PP_resNet/"

In [ ]:
# Import Data 
with open('/content/drive/MyDrive/NSF_Research/NN/Data/fDOM_data', 'rb') as f: fDOM_data = pickle.load(f)
with open('/content/drive/MyDrive/NSF_Research/NN/Data/turb_data', 'rb') as f: turb_data = pickle.load(f)
with open('/content/drive/MyDrive/NSF_Research/NN/Data/stage_data', 'rb') as f: stage_data = pickle.load(f)

# K-Fold Cross Validation w/ FCN + 1 Dense Layer

In [ ]:
# Define Parameters
num_splits = 5
batch_size = 32
epochs = 200
overfit_patience = 15

accumulated_test_metrics = []
accumulated_cfmxs = {}

# Get unformatted data
X,y = create_turbPP_dataset(fDOM_data, stage_data, turb_data, window_size = 10)

# Reshape and format to make train and test set for deep learning models
X = np.array(X)
y = np.array(y)
print(X[0].shape)
input_shape = X[0].shape
num_classes = len(np.unique(y, axis = 0))
encoder = OneHotEncoder(categories='auto')
encoder.fit(y.reshape(-1,1))
y = encoder.transform(y.reshape(-1,1)).toarray()
y_true = np.argmax(y, axis =1)

train_test_split_indices = TimeSeriesSplit(num_splits).split(X)

In [ ]:
split = 1

for train_val_indices, test_indices in train_test_split_indices:
  print("Split: ", split)
  X_train, y_train = np.take(X,train_val_indices, axis = 0), np.take(y,train_val_indices, axis = 0)

  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .10)
  X_test, y_test = np.take(X, test_indices, axis = 0), np.take(y,test_indices, axis = 0)

  model = ResNet(output_directory, input_shape, num_classes,overfit_patience)
  model.fit(X_train,y_train,X_val,y_val,batch_size,epochs)
  test_pred= model.predict(X_test)

  cfmx = confusion_matrix(np.argmax(y_test, axis = 1),np.argmax(test_pred, axis = 1))
  accumulated_cfmxs[split] = copy.deepcopy(cfmx)

  test_ba = metrics.balanced_accuracy_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1))
  test_f1 = metrics.f1_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1))
  TN, FP, FN, TP = cfmx.ravel()
  test_tpr = TP/(TP+FN)
  test_tnr = TN/(TN+FP)
  test_precision = TP/(TP+FP) 

  print("Test F1: ", test_f1, " Test BA: ", test_ba)
  accumulated_test_metrics.append({'f1' : test_f1, 'ba': test_ba, 'precision': test_precision, 'tpr': test_tpr, 'tnr': test_tnr})

  split +=1 


# Display Resuls 
mean_f1 = 0
mean_ba = 0 
mean_sensitivity = 0
mean_specificity = 0 
mean_precision = 0

for split in accumulated_test_metrics:
  
    mean_f1+=split['f1']
    mean_ba+=split['ba']
    mean_sensitivity += split['tpr']
    mean_specificity += split['tnr']
    mean_precision += split['precision']

print("Mean Test F1: ", mean_f1/len(accumulated_test_metrics))
print("Mean Test BA: ", mean_ba/len(accumulated_test_metrics))
print("Mean Test TPR: ", mean_sensitivity/len(accumulated_test_metrics))
print("Mean Test TNR: ", mean_specificity/len(accumulated_test_metrics))
print("Mean Test Precision: ", mean_precision/len(accumulated_test_metrics))

mean_cfmx = np.zeros((2,2))
for key in accumulated_cfmxs.keys():
    mean_cfmx += accumulated_cfmxs[key]
mean_cfmx = mean_cfmx / num_splits

plt.figure(figsize = (10,7))
plt.title(label = 'Turbidity Phantom Peak (ResNet)')

sn.set(font_scale = 1.5)
sn.heatmap( pd.DataFrame(mean_cfmx.astype('float') / mean_cfmx.sum(axis=1)[:, np.newaxis],index = ['Negative', 'Positive'], columns = ['Negative','Positive']), annot = True, annot_kws ={"size": 16})
plt.xlabel('Ground Truths')
plt.ylabel('Predictions')
plt.show()

plt.figure(figsize = (10,7))
plt.title(label = 'Turbidity Phantom Peak (ResNet)')

sn.set(font_scale = 1.5)
sn.heatmap( pd.DataFrame(mean_cfmx,index = ['Negative', 'Positive'], columns = ['Negative','Positive']), annot = True, annot_kws ={"size": 16})
plt.xlabel('Ground Truths')
plt.ylabel('Predictions')
plt.show()

In [ ]:
# Fully Convolutional Network ... that I have corrupted by adding fully connected layers at the end
class FCN:

  def __init__(self, output_directory, input_shape, nb_classes, overfit_patience):
    self.output_directory = output_directory
    self.model = self.build_model(input_shape, nb_classes, overfit_patience)
    print(self.model.summary())
    self.model.save_weights(self.output_directory+'model_init.hdf5')
    return


  def build_model(self, input_shape, nb_classes, overfit_patience):
    padding = 'same'
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128,kernel_size=3,padding=padding)(input_layer)
    conv1 = keras.layers.Activation('relu')(conv1)
    conv1 = keras.layers.AveragePooling1D(pool_size=2)(conv1)

    conv2 = keras.layers.Conv1D(filters=256,kernel_size=3,padding=padding)(conv1)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.AveragePooling1D(pool_size=2)(conv2)

    conv3 = keras.layers.Conv1D(filters=128,kernel_size=3,padding=padding)(conv2)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.AveragePooling1D(pool_size=2)(conv3)
    

    flatten_layer = keras.layers.Flatten()(conv3)
    
    fc1 = keras.layers.Dense(units = 64, activation = 'relu')(flatten_layer)
    fc2 = keras.layers.Dense(units = 16, activation = 'relu')(fc1)

    output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(fc2)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    overfit_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience= overfit_patience,verbose = True)

    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), metrics=['accuracy'])

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, 
      min_lr=0.0001)

    file_path = self.output_directory+'best_model.hdf5'

    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', 
      save_best_only=True)

    self.callbacks = [reduce_lr,model_checkpoint, overfit_callback]

    return model 

  def fit(self, x_train, y_train, x_val, y_val, batch_size, epochs):
    
    mini_batch_size = int(min(x_train.shape[0]/10, batch_size))

    hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=epochs,
      verbose=False, validation_data=(x_val,y_val), callbacks=self.callbacks)
    
    self.model.save(self.output_directory+'last_model.hdf5')

    keras.backend.clear_session()

  def predict(self, X):
    model_path = self.output_directory + 'best_model.hdf5'
    model = keras.models.load_model(model_path)
    return model.predict(X)
    
		

In [ ]:
class ResNet:

  def __init__(self, output_directory, input_shape, nb_classes, overfit_patience):
    self.output_directory = output_directory
    self.model = self.build_model(input_shape, nb_classes,64,overfit_patience)
    # print(self.model.summary())
    self.model.save_weights(self.output_directory+'model_init.hdf5')
    return
  
  

  def build_model(self, input_shape, num_classes, num_feature_maps, overfit_patience):
  
    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1

    conv_x = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # BLOCK 2

    conv_x = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # # BLOCK 3

    # conv_x = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
    # conv_x = keras.layers.BatchNormalization()(conv_x)
    # conv_x = keras.layers.Activation('relu')(conv_x)

    # conv_y = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    # conv_y = keras.layers.BatchNormalization()(conv_y)
    # conv_y = keras.layers.Activation('relu')(conv_y)

    # conv_z = keras.layers.Conv1D(filters=num_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    # conv_z = keras.layers.BatchNormalization()(conv_z)

    # # no need to expand channels because they are equal
    # shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    # output_block_3 = keras.layers.add([shortcut_y, conv_z])
    # output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # FINAL

    flatten_layer = keras.layers.Flatten()(output_block_2)
    
    fc1 = keras.layers.Dense(units = 64, activation = 'relu')(flatten_layer)
    fc2 = keras.layers.Dense(units = 16, activation = 'relu')(fc1)

    output_layer = keras.layers.Dense(units=num_classes,activation='softmax')(fc2)

    # output_layer = keras.layers.Dense(num_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=overfit_patience, min_lr=0.0001)

    overfit_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience= overfit_patience,verbose = True)

    file_path = self.output_directory + 'best_model.hdf5'

    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                        save_best_only=True)

    self.callbacks = [reduce_lr, model_checkpoint,overfit_callback]

    return model

  def fit(self, x_train, y_train, x_val, y_val, batch_size, epochs):
    
    mini_batch_size = int(min(x_train.shape[0]/10, batch_size))

    hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=epochs,
      verbose=False, validation_data=(x_val,y_val), callbacks=self.callbacks)
    
    self.model.save(self.output_directory+'last_model.hdf5')



    # convert the predicted from binary to integer 
    

    # save_logs(self.output_directory, hist, y_pred, y_true, duration)

    keras.backend.clear_session()

  def predict(self, X):
    model_path = self.output_directory + 'best_model.hdf5'
    model = keras.models.load_model(model_path)
    return model.predict(X)

In [ ]:
def create_turbPP_dataset(fDOM_data, stage_data, turb_data, window_size = 15):
    
    # Get turb candidate peaks - these were the values used to get the set of turb candidate peaks
    params = {'prom' : [6,None],
                    'width': [None, None],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .6}
    
    turb_peaks, turb_props = find_peaks(turb_data[:,1], 
                                        prominence = params['prom'],
                                        width = params['width'],
                                        wlen = params['wlen'],
                                        distance = params['dist'],
                                        rel_height = params['rel_h'])
    
    turb_peaks, turb_props = delete_missing_data_peaks(turb_data, turb_peaks, turb_props, '/content/drive/MyDrive/NSF_Research/NN/Data/flat_plat_ranges.txt')

    # Import ground truths for these peaks
    gt_fname_t = '/content/drive/MyDrive/NSF_Research/NN/Data/turb_pp_0k-300k-2_labeled'
    with open(gt_fname_t, 'r', newline = '') as f:
        reader = csv.reader(f, delimiter = ',')
        # gt entries in form: ['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak']
        next(reader)
        ground_truth = [0 if row[2] == 'NPP' else 1 for row in reader] 
        f.close()  

    # Reshape data 
    fDOM_data = fDOM_data[:,1].reshape(-1,1)
    stage_data = stage_data[:,1].reshape(-1,1)
    turb_data = turb_data[:,1].reshape(-1,1)
                                         
        
    # Use Robust scaler to scale data
    fDOM_scaler = RobustScaler().fit(fDOM_data)
    fDOM_data_scaled = fDOM_scaler.transform(fDOM_data)
    
    turb_scaler = RobustScaler().fit(turb_data)
    turb_data_scaled = turb_scaler.transform(turb_data)
    
    stage_scaler = RobustScaler().fit(stage_data)
    stage_data_scaled = stage_scaler.transform(stage_data)
    
    # Created "sequenced" data, where middle point is the peak --> sequence is (window_size * 2) + 1
    X = []
    y = []
    
    for i,peak in enumerate(turb_peaks):
        if peak - window_size > 0 and peak + window_size < len(fDOM_data): 
            sample = np.hstack((fDOM_data_scaled[peak - window_size:peak + window_size + 1],
                                stage_data_scaled[peak - window_size:peak + window_size + 1],
                                turb_data_scaled[peak - window_size:peak + window_size + 1])).T
            y.append(ground_truth[i])
            X.append(sample.T)
    
    return X, y


In [ ]:
def datetime_to_julian(date : datetime.datetime) -> float:
    """
    Convert datetime object to julian time using algorithm outlined by Fliegel and van Flandern (1968):
    
    date:   date to convert

    return: julian time equivalent
    """
    interm1 = math.floor((14-date.month)/12)
    interm2 = date.year + 4800 - interm1
    interm3 = date.month + 12*interm1 - 3

    jdn = (date.day + math.floor((153*interm3 + 2)/5) + 365*interm2 + 
           math.floor(interm2/4) - math.floor(interm2/100) + math.floor(interm2/400) - 32045)

    jd = jdn + (date.hour - 12) / 24 + date.minute / 1440 + date.second / 86400 + date.microsecond / 86400000000
    return jd

def delete_missing_data_peaks(data, peaks, props, missing_file_path):
    """ 
    Delete peaks that occur during time periods designated as "missing data"
    
    data:              timeseries that peaks occured in
    peaks:             indices of peaks detected
    props:             properties associated with each peak
    missing_file_path: file path of missing date ranges
    return:            filtered peaks and props
    """
    with open(missing_file_path,newline='') as file:
        reader = csv.reader(file, delimiter = ',')
        time_list = []
        for row in reader:
            time_list.append([datetime_to_julian(datetime.datetime.strptime(row[0],'%Y-%m-%d %H:%M:%S')),
                              datetime_to_julian(datetime.datetime.strptime(row[1],'%Y-%m-%d %H:%M:%S'))])
            
        # Identify and remove violating peaks 
        keep_indices = list(np.linspace(0,peaks.shape[0]-1,peaks.shape[0]))
        for i,idx in enumerate(peaks): 
            time = data[idx,0]
            for row in time_list: 
                if time >= row[0] and time <= row[1]:
                    keep_indices.remove(i)
                    break
        
        peaks = np.take(peaks,keep_indices,0)  
        
        # Remove properties for violating peaks
        for key in props:
            props[key] = np.take(props[key], keep_indices,0)
        
        return peaks, props